# Testing `SeqData`

**Authorship:**
Adam Klie, *04/06/2022*
***
**Description:**
Notebook to test the EUGENE data structure `SeqData` 
***
<div class="alert alert-block alert-warning">
<b>TODOs</b>:
<ul>
    <b><li></li></b>
    <b><li></li></b>
    <b><li></li></b>
    </ul>
</div>

# Setup

In [7]:
# Classics
import numpy as np
import pandas as pd

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

# Basic import
import eugene as eu
import logging
eu.settings.verbosity = logging.INFO

Global seed set to 13


## Test `SeqData` on random dataset

In [8]:
random1000 = eu.datasets.random1000()
random1000

SeqData object with = 1000 seqs
seqs = (1000,)
names = (1000,)
rev_seqs = None
ohe_seqs = None
ohe_rev_seqs = None
    seqs_annot: 'TARGETS'

In [9]:
eu.pp.reverse_complement_data(random1000, copy=False)

SeqData object modified:
	rev_seqs: None -> 1000 rev_seqs added


In [10]:
eu.pp.one_hot_encode_data(random1000, copy=False)

SeqData object modified:
	ohe_seqs: None -> 1000 ohe_seqs added
	ohe_rev_seqs: None -> 1000 ohe_rev_seqs added


In [11]:
random1000

SeqData object with = 1000 seqs
seqs = (1000,)
names = (1000,)
rev_seqs = (1000,)
ohe_seqs = (1000, 66, 4)
ohe_rev_seqs = (1000, 66, 4)
    seqs_annot: 'TARGETS'

In [27]:
eu.pp.train_test_split_data(random1000, copy=False, kwargs={})

800


In [29]:
random1000.seqs_annot["TRAIN"]

0       True
1       True
2       True
3      False
4      False
       ...  
995     True
996     True
997     True
998     True
999     True
Name: TRAIN, Length: 1000, dtype: bool

# Reading and writing

In [6]:
sdata = eu.dl.read_csv(file="../_data/test_1000seqs_66/test_seqs.tsv")
sdata

SeqData object with = 1000 seqs
seqs = (1000,)
names = None
rev_seqs = None
ohe_seqs = None
ohe_rev_seqs = None
    seqs_annot: 'TARGETS'

In [7]:
sdata = eu.dl.read_fasta(seq_file="../_data/test_1000seqs_66/test_seqs.fa", target_file="../_data/test_1000seqs_66/test_activities.npy")
sdata

SeqData object with = 1000 seqs
seqs = (1000,)
names = (1000,)
rev_seqs = None
ohe_seqs = None
ohe_rev_seqs = None
    seqs_annot: 'TARGETS'

In [8]:
sdata = eu.dl.read_numpy(seq_file="../_data/test_1000seqs_66/test_ohe_seqs.npy", names_file="../_data/test_1000seqs_66/test_ids.npy", target_file="../_data/test_1000seqs_66/test_activities.npy", ohe_encoded=True)
sdata.seqs = np.load("../_data/test_1000seqs_66/test_seqs.npy")
sdata.write_h5sd("../_data/test_1000seqs_66/test_seqs.h5sd")
sdata

SeqData object with = 1000 seqs
seqs = (1000,)
names = (1000,)
rev_seqs = None
ohe_seqs = (1000, 66, 4)
ohe_rev_seqs = None
    seqs_annot: 'TARGETS'

In [9]:
sdata = eu.dl.read("../_data/test_1000seqs_66/test_seqs.h5sd")
sdata

SeqData object with = 1000 seqs
seqs = (1000,)
names = (1000,)
rev_seqs = None
ohe_seqs = (1000, 66, 4)
ohe_rev_seqs = None
    seqs_annot: 'TARGETS'

# Creating a dataloader

In [40]:
sdataset = sdata.to_dataset(label="TARGETS", seq_transforms = ["augment", "one_hot_encode"], transform_kwargs={"enhancer": "Core-otx-a"})

In [41]:
sdataset.seqs[0], sdataset.targets[0], sdataset.rev_seqs, sdataset.transform

('GTAGGTAAGCGGGGTATTTGCACTTCCCTTAATCCATAAGGGCTTTTGCCGCGTGTTAGAGGAAGC',
 0.7708982213861482,
 None,
 Compose(
 ))

In [42]:
sdataset[0]

(tensor([115., 101., 113.,  48.,  48.,  49.,  36.]),
 tensor([[0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 0., 1.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
  

In [43]:
# Instantiate a DataLoader
from torch.utils.data import DataLoader
test_dataloader = DataLoader(sdataset, batch_size=32, shuffle=True, num_workers=0)

In [45]:
# Check the DataLoader
for i_batch, sample_batched in enumerate(test_dataloader):
    print(i_batch, sample_batched[0].size(), sample_batched[1].size(), sample_batched[2].size(), sample_batched[3].size())
    # observe 4th batch and stop.
    if i_batch == 3:
        break

0 torch.Size([32, 7]) torch.Size([32, 66, 4]) torch.Size([32, 1]) torch.Size([32])
1 torch.Size([32, 7]) torch.Size([32, 66, 4]) torch.Size([32, 1]) torch.Size([32])
2 torch.Size([32, 7]) torch.Size([32, 66, 4]) torch.Size([32, 1]) torch.Size([32])
3 torch.Size([32, 7]) torch.Size([32, 66, 4]) torch.Size([32, 1]) torch.Size([32])


# SeqDataModule

In [49]:
from torchvision import transforms
data_transform = transforms.Compose([eu.dl.ToTensor(transpose=True)])
datamodule = eu.dl.SeqDataModule(seq_file=f"../_data/test_1000seqs_66/test_ohe_seqs.npy", batch_size=32, transform=data_transform, read_kwargs={"return_numpy": True})
datamodule.setup()
dataset = datamodule.train_dataloader().dataset
assert(len(dataset[0]) == 4)

In [50]:
dataset[0]

(tensor([-1.]),
 tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
          0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0.,
          0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.,
          0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
          1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
          0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
         [1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,

---

# Scratch

In [6]:
names, seqs, rev_seqs, targets = eu.dl.read_numpy(f"../_data/test_1000seqs_66/test_seqs.npy", names_file=f"../_data/test_1000seqs_66/test_ids.npy", target_file=f"../_data/test_1000seqs_66/test_labels.npy", rev_seq_file=f"../_data/test_1000seqs_66/test_rev_seqs.npy", return_numpy=True)
sdata = eu.dl.SeqData(names=names, seqs=seqs, seqs_annot=targets, rev_seqs=rev_seqs)
sdataset = sdata.to_dataset(label=0, seq_transforms = ["augment", "one_hot_encode"], transform_kwargs={"enhancer": "Core-otx-a"})
assert(sdataset[0])

In [5]:
    names, seqs, rev_seqs, targets = eu.dl.read_numpy("../_data/test_1000seqs_66/test_seqs.npy", names_file="../_data/test_1000seqs_66/test_ids.npy", target_file="../_data/test_1000seqs_66/test_labels.npy", rev_seq_file="../_data/test_1000seqs_66/test_rev_seqs.npy", return_numpy=True)
    assert(len(names) == len(seqs) == len(rev_seqs) == len(targets))
    sdata = eu.dl.SeqData(names=names, seqs=seqs, seqs_annot=targets, rev_seqs=rev_seqs)

/mnt/beegfs/users/aklie/projects/EUGENE/eugene/dataloading/dataloaders/_SeqData.py:220: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if index_name in anno:


In [6]:
sdata.seqs_annot

,0
0,1
1,0
2,1
3,0
4,0
...,...
995,0
996,0
997,0
998,0


In [8]:
sdata.write_h5sd("../_data/test_1000seqs_66/test_seqs.h5sd")

In [ ]:
def write_elem(
    f: "Union[H5Group, ZarrGroup]",
    k: str,
    elem: Any,
    *args,
    modifiers=frozenset(),
    **kwargs,
):
    """
    Write an element to a disk store using it's anndata encoding.
    Params
    ------
    f
        The store to write to.
    k
        The key to write for this value.
    elem
        The element to write as k to f.
    """
    dest_type = type(f)
    if elem is None:
        return
    t = type(elem)
    if k == "/":
        f.clear()
    elif k in f:
        del f[k]
    if (
        hasattr(elem, "dtype")
        and (dest_type, (t, elem.dtype.kind), modifiers) in write
    ):
        get_writer(dest_type, (t, elem.dtype.kind), modifiers)(
            f, k, elem, *args, **kwargs
        )
    else:
        get_writer(dest_type, t, modifiers)(f, k, elem, *args, **kwargs)

In [ ]:
    def get_writer(self, dest_type, typ, modifiers=frozenset()):
        import h5py

        if dest_type is h5py.File:
            dest_type = h5py.Group
        modifiers = frozenset(modifiers)

        if (dest_type, typ, modifiers) not in self.write:
            raise TypeError(
                f"No method has been defined for writing {typ} elements to {dest_type}"
            )

        return self.write[(dest_type, typ, modifiers)]

In [ ]:
from typing import Any, NamedTuple, Tuple, Type, Callable, Union

In [ ]:
write_elem(f, "seq_annot", sdata.seqs_annot)

AttributeError: type object 'Group' has no attribute 'write'

In [ ]:
sdata.write_h5sd(filepath)

In [ ]:
test_dict = eu.datasets.load_h5sd(filepath)

{'seqs': array(['...'],
      dtype='<U66'), 'names': array(['seq001', 'seq002', 'seq003', 'seq004', 'seq005', 'seq006',
       'seq007', 'seq008', 'seq009', 'seq010', 'seq011', 'seq012',
       'seq013', 'seq014', 'seq015', 'seq016', 'seq017', 'seq018',
       'seq019', 'seq020', 'seq021', 'seq022', 'seq023', 'seq024',
       'seq025', 'seq026', 'seq027', 'seq028', 'seq029', 'seq030',
       'seq031', 'seq032', 'seq033', 'seq034', 'seq035', 'seq036',
       'seq037', 'seq038', 'seq039', 'seq040', 'seq041', 'seq042',
       'seq043', 'seq044', 'seq045', 'seq046', 'seq047', 'seq048',
       'seq049', 'seq050', 'seq051', 'seq052', 'seq053', 'seq054',
       'seq055', 'seq056', 'seq057', 'seq058', 'seq059', 'seq060',
       'seq061', 'seq062', 'seq063', 'seq064', 'seq065', 'seq066',
       'seq067', 'seq068', 'seq069', 'seq070', 'seq071', 'seq072',
       'seq073', 'seq074', 'seq075', 'seq076', 'seq077', 'seq078',
       'seq079', 'seq080', 'seq081', 'seq082', 'seq083', 'seq084',
       '

In [ ]:
test_dict.ohe_seqs

array([[[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        ...,
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.]],

       [[0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        ...,
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]],

       [[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.]],

       ...,

       [[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        ...,
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]],

       [[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        ...,
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        ...,
        [1., 0., 0., 0.],
        [0., 0.

In [ ]:
with h5py.File(filepath, "r") as f:
    print(f.name)
    print(list(f.keys()))
    print(f["seqs"].shape)
    print(f["seqs"][0].decode('latin-1'))
    print(f["seqs_annot"].attrs["label"][0])

/
['names', 'ohe_seqs', 'seqs', 'seqs_annot']
(1000,)
GTAGGTAAGCGGGGTATTTGCACTTCCCTTAATCCATAAGGGCTTTTGCCGCGTGTTAGAGGAAGC


KeyError: "Can't open attribute (can't locate attribute: 'label')"

In [ ]:
with h5py.File(filename, "r") as f:
        d = {}
        for k in f.keys():
            # Backwards compat for old raw
            if k == "raw" or k.startswith("raw."):
                continue
            if k == "X" and "X" in as_sparse:
                d[k] = rdasp(f[k])
            elif k == "raw":
                assert False, "unexpected raw format"
            elif k in {"obs", "var"}:
                # Backwards compat
                d[k] = read_dataframe(f[k])
            else:  # Base case
                d[k] = read_elem(f[k])

        d["raw"] = _read_raw(f, as_sparse, rdasp)

        X_dset = f.get("X", None)
        if X_dset is None:
            pass
        elif isinstance(X_dset, h5py.Group):
            d["dtype"] = X_dset["data"].dtype
        elif hasattr(X_dset, "dtype"):
            d["dtype"] = f["X"].dtype
        else:
            raise ValueError()

In [ ]:
h5_read = h5py.File(filepath, "r")

In [ ]:
h5_read.attrs["encoding-type"]

'Seqdata'

---